# Google Driveマウント・ディレクトリの準備

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd "/content/drive/My Drive/Colab Notebooks/graph_network"

/content/drive/My Drive/Colab Notebooks/graph_network


In [ ]:
%mkdir -p tmp/datasets
%mkdir -p tmp/models
%mkdir -p tmp/rollouts

In [3]:
%ls -l

total 46
-rw------- 1 root root 38161 Mar 29 13:35 graph_network.ipynb
drwx------ 2 root root  4096 Mar 22 02:39 learning_to_simulate/
drwx------ 2 root root  4096 Mar 29 03:22 tmp/


# モジュールのインストール

In [4]:
!pip install -r learning_to_simulate/requirements.txt

     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 110.5 MB 1.4 kB/s 
     |████████████████████████████████| 665 kB 62.9 MB/s 
     |████████████████████████████████| 2.5 MB 61.9 MB/s 
     |████████████████████████████████| 20.1 MB 775 kB/s 
     |████████████████████████████████| 50 kB 9.4 MB/s 
     |████████████████████████████████| 503 kB 61.5 MB/s 
     |████████████████████████████████| 2.9 MB 53.4 MB/s 
     |████████████████████████████████| 3.8 MB 59.8 MB/s 
  Created wheel for graph-nets: filename=graph_nets-1.1.0-py3-none-any.whl size=91857 sha256=e258c27795f46d605d91027a046a608aac0cadb501fca5fa47269dd62029aaf4
  Stored in directory: /root/.cache/pip/wheels/44/3d/65/f2e8f0a8d0b28bea5f168fc717261a67303d2183a3e450c812
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=6837848544022e7fd6fcaaf54430adefe6d578504ebbe7c05abfdcd4ee4f2c88
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d09

In [10]:
import tensorflow as tf
record = next(tf.python_io.tf_record_iterator('tmp/datasets/WaterRamps/test.tfrecord'))
print(tf.train.Example.FromString(record))

features {
  feature {
    key: "key"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "particle_type"
    value {
      bytes_list {
        value: "\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\005\000\000

# データセットのダウンロード

In [ ]:
!bash ./learning_to_simulate/download_dataset.sh WaterRamps tmp/datasets

# 学習

In [ ]:
!python -m learning_to_simulate.train --data_path=tmp/datasets/WaterRamps --model_path=tmp/models/WaterRamps

# ロールアウト

In [ ]:
!python -m learning_to_simulate.train --mode="eval_rollout" --data_path=tmp/datasets/WaterRamps --model_path=tmp/models/WaterRamps --output_path=tmp/rollouts/WaterRamps

I0329 06:17:50.911465 140477205227392 train.py:469] Saving: tmp/rollouts/WaterRamps/rollout_test_0.pkl.
I0329 06:33:21.626461 140477205227392 train.py:469] Saving: tmp/rollouts/WaterRamps/rollout_test_1.pkl.
I0329 06:46:38.652303 140477205227392 train.py:469] Saving: tmp/rollouts/WaterRamps/rollout_test_2.pkl.
I0329 07:22:16.777964 140477205227392 train.py:469] Saving: tmp/rollouts/WaterRamps/rollout_test_3.pkl.
I0329 07:26:28.725491 140477205227392 train.py:469] Saving: tmp/rollouts/WaterRamps/rollout_test_4.pkl.
^C


# 可視化

In [ ]:
import pickle

from absl import app
from absl import flags

from matplotlib import animation
from matplotlib import rc
import matplotlib.pyplot as plt
import numpy as np


rollout_path = 'tmp/rollouts/WaterRamps/rollout_test_0.pkl'
step_stride = 3
block_on_show = True

TYPE_TO_COLOR = {
    3: "black",  # Boundary particles.
    0: "green",  # Rigid solids.
    7: "magenta",  # Goop.
    6: "gold",  # Sand.
    5: "blue",  # Water.
}

with open(rollout_path, "rb") as file:
    rollout_data = pickle.load(file)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
plot_info = []
for ax_i, (label, rollout_field) in enumerate(
    [("Ground truth", "ground_truth_rollout"),
     ("Prediction", "predicted_rollout")]):
    # Append the initial positions to get the full trajectory.
    trajectory = np.concatenate([
        rollout_data["initial_positions"],
        rollout_data[rollout_field]], axis=0)
    ax = axes[ax_i]
    ax.set_title(label)
    bounds = rollout_data["metadata"]["bounds"]
    ax.set_xlim(bounds[0][0], bounds[0][1])
    ax.set_ylim(bounds[1][0], bounds[1][1])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1.)
    points = {
        particle_type: ax.plot([], [], "o", ms=2, color=color)[0]
        for particle_type, color in TYPE_TO_COLOR.items()}
    plot_info.append((ax, trajectory, points))

num_steps = trajectory.shape[0]

def update(step_i):
    outputs = []
    for _, trajectory, points in plot_info:
        for particle_type, line in points.items():
            mask = rollout_data["particle_types"] == particle_type
            line.set_data(trajectory[step_i, mask, 0],
                          trajectory[step_i, mask, 1])
            outputs.append(line)
    return outputs

unused_animation = animation.FuncAnimation(
    fig, update,
    frames=np.arange(0, num_steps, step_stride), interval=10)
rc('animation', html='jshtml')
unused_animation
# plt.show(block=block_on_show)

Output hidden; open in https://colab.research.google.com to view.

# 可視化 (no use)

In [ ]:
!python -m learning_to_simulate.render_rollout_tanabe --rollout_path=tmp/rollouts/WaterRamps/rollout_test_0.pkl